In [50]:
import numpy as np

seed = 1
gen = np.random.default_rng(seed=seed)

n = 5
# 2^n random numbers
v = gen.uniform(0, 1, 2**n) # set function
p = gen.uniform(0, 1, n) # probability distribution

def compute_semivalue(v, p, i, n):
    semivalue_i = 0
    for set_idx in range(2**n):
        bin_string = bin(set_idx)[2:].zfill(n)
        size = bin_string.count('1')
        if bin_string[i] == '0':
            semivalue_i -= p[size] * v[set_idx]
        else:
            semivalue_i += p[size-1] * v[set_idx]
    return semivalue_i


In [73]:
A = np.zeros((2**n, n))
# TODO: COMPUTE a_n
for set_idx in range(2**n):
    bin_string = bin(set_idx)[2:].zfill(n)
    binary_vec = np.array([int(x) for x in bin_string])
    size = np.sum(binary_vec)
    ps = p[size] if size < n else 0
    ps1 = p[size-1] if size > 0 else 0

    A[set_idx] = binary_vec * ps - (1-binary_vec) * ps1

# Solve argmin_x ||Ax - v||_2
x = np.linalg.lstsq(A, v, rcond=None)[0]
semivalues = [compute_semivalue(v, p, i, n) for i in range(n)]

In [72]:
i, j = 0, 2
semivalues[i] - semivalues[j], x[i] - x[j]

(-0.39325514926091054, -0.025055643850584286)